# Snowflake ML Demo: Analytics Tables Setup

This notebook creates the analytics infrastructure and ML model tracking tables needed for our machine learning pipeline.

## What We're Creating

### Analytics Tables (DEMO_ANALYTICS schema)
- **PATIENT_RISK_SCORES**: Store patient risk assessments
- **DRUG_INTERACTION_ALERTS**: Track potential drug interactions
- **AE_PREDICTIONS**: Log model predictions and results
- **POPULATION_VS_FAERS_COMPARISON**: Compare local vs national adverse event rates

### ML Model Tables (ML_MODELS schema)
- **MODEL_REGISTRY**: Track model versions and performance
- **FEATURE_IMPORTANCE**: Store feature importance scores
- **MODEL_PREDICTIONS_LOG**: Detailed prediction logging

## Purpose
These tables provide the infrastructure for model deployment, monitoring, and business analytics.

## Prerequisites
- Running in Snowflake Notebooks environment
- Environment setup completed (01_Environment_Setup notebook)
- FAERS data setup completed (02_FAERS_Data_Setup notebook)


In [ ]:
# Initialize Snowflake Session
print("Initializing Snowflake session...")

# Import Snowpark session (available in Snowflake Notebooks)
from snowflake.snowpark.context import get_active_session

# Get the active Snowflake session
session = get_active_session()

print("SUCCESS: Snowflake session initialized!")

# Set context for analytics tables setup
print("Setting context for analytics tables...")
session.sql("USE DATABASE ADVERSE_EVENT_MONITORING").collect()
session.sql("USE WAREHOUSE ADVERSE_EVENT_WH").collect()

# Verify context
current_context = session.sql("""
    SELECT 
        CURRENT_DATABASE() as database,
        CURRENT_WAREHOUSE() as warehouse
""").collect()[0]

print(f"   Database: {current_context['DATABASE']}")
print(f"   Warehouse: {current_context['WAREHOUSE']}")
print("SUCCESS: Context set successfully!")


In [ ]:
# Create analytics tables in DEMO_ANALYTICS schema
print("Creating analytics tables in DEMO_ANALYTICS schema...")

# Switch to DEMO_ANALYTICS schema
session.sql("USE SCHEMA DEMO_ANALYTICS").collect()
print("SUCCESS: Switched to DEMO_ANALYTICS schema")


In [ ]:
# Create PATIENT_RISK_SCORES table
print("Creating PATIENT_RISK_SCORES table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS PATIENT_RISK_SCORES (
        patient_id VARCHAR(50),
        risk_score FLOAT,
        risk_category VARCHAR(20),
        primary_risk_factors ARRAY,
        medication_count INTEGER,
        condition_count INTEGER,
        age_group VARCHAR(20),
        calculation_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: PATIENT_RISK_SCORES table created successfully!")


In [ ]:
# Create DRUG_INTERACTION_ALERTS table
print("Creating DRUG_INTERACTION_ALERTS table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS DRUG_INTERACTION_ALERTS (
        alert_id VARCHAR(50),
        patient_id VARCHAR(50),
        drug1 VARCHAR(200),
        drug2 VARCHAR(200),
        interaction_type VARCHAR(100),
        severity_level VARCHAR(20),
        clinical_significance VARCHAR(500),
        recommendation VARCHAR(1000),
        alert_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: DRUG_INTERACTION_ALERTS table created successfully!")

# Create AE_PREDICTIONS table
print("Creating AE_PREDICTIONS table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS AE_PREDICTIONS (
        prediction_id VARCHAR(50),
        patient_id VARCHAR(50),
        predicted_ae VARCHAR(200),
        probability FLOAT,
        confidence_interval VARCHAR(50),
        model_version VARCHAR(20),
        prediction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: AE_PREDICTIONS table created successfully!")


In [ ]:
# Create POPULATION_VS_FAERS_COMPARISON table
print("Creating POPULATION_VS_FAERS_COMPARISON table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS POPULATION_VS_FAERS_COMPARISON (
        medication VARCHAR(200),
        adverse_event VARCHAR(200),
        local_population_rate FLOAT,
        faers_national_rate FLOAT,
        rate_ratio FLOAT,
        statistical_significance FLOAT,
        sample_size INTEGER,
        analysis_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: POPULATION_VS_FAERS_COMPARISON table created successfully!")


In [ ]:
# Create ML model tracking tables in ML_MODELS schema
print("Creating ML model tracking tables in ML_MODELS schema...")

# Switch to ML_MODELS schema
session.sql("USE SCHEMA ML_MODELS").collect()
print("SUCCESS: Switched to ML_MODELS schema")

# Create MODEL_REGISTRY table
print("Creating MODEL_REGISTRY table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS MODEL_REGISTRY (
        model_id VARCHAR(50),
        model_name VARCHAR(100),
        model_type VARCHAR(50),
        model_version VARCHAR(20),
        training_date TIMESTAMP,
        accuracy_score FLOAT,
        precision_score FLOAT,
        recall_score FLOAT,
        f1_score FLOAT,
        model_status VARCHAR(20),
        created_by VARCHAR(100)
    )
""").collect()

print("SUCCESS: MODEL_REGISTRY table created successfully!")


In [ ]:
# Create FEATURE_IMPORTANCE table
print("Creating FEATURE_IMPORTANCE table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS FEATURE_IMPORTANCE (
        model_id VARCHAR(50),
        feature_name VARCHAR(100),
        importance_score FLOAT,
        feature_type VARCHAR(50),
        created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: FEATURE_IMPORTANCE table created successfully!")

# Create MODEL_PREDICTIONS_LOG table
print("Creating MODEL_PREDICTIONS_LOG table...")

session.sql("""
    CREATE TABLE IF NOT EXISTS MODEL_PREDICTIONS_LOG (
        prediction_id VARCHAR(50),
        model_id VARCHAR(50),
        patient_id VARCHAR(50),
        input_features OBJECT,
        prediction_result OBJECT,
        confidence_score FLOAT,
        prediction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("SUCCESS: MODEL_PREDICTIONS_LOG table created successfully!")


In [ ]:
# Verify analytics tables were created successfully
print("Verifying DEMO_ANALYTICS tables were created...")

# Switch to DEMO_ANALYTICS schema and show tables
session.sql("USE SCHEMA DEMO_ANALYTICS").collect()
analytics_tables = session.sql("SHOW TABLES").collect()

print("\nDEMO_ANALYTICS Tables:")
for table in analytics_tables:
    print(f"   - {table[1]}")  # table[1] contains the table name

print(f"\nSUCCESS: {len(analytics_tables)} analytics tables created successfully!")


In [ ]:
# Verify ML model tables were created successfully
print("Verifying ML_MODELS tables were created...")

# Switch to ML_MODELS schema and show tables
session.sql("USE SCHEMA ML_MODELS").collect()
ml_tables = session.sql("SHOW TABLES").collect()

print("\nML_MODELS Tables:")
for table in ml_tables:
    print(f"   - {table[1]}")  # table[1] contains the table name

print(f"\nSUCCESS: {len(ml_tables)} ML model tables created successfully!")
print("\nAll analytics infrastructure is ready!")


In [ ]:
# Create Healthcare Analytics Data (Required for Notebook 4)
print("Creating healthcare analytics data for feature engineering...")

# Generate sample healthcare data for ML training
session.sql("""
CREATE OR REPLACE TABLE ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.PREPARED_HEALTHCARE_ANALYTICS AS
WITH patient_base AS (
    SELECT 
        CONCAT('PAT_', LPAD(SEQ4(), 7, '0')) as PATIENT_ID,
        FLOOR(UNIFORM(18, 90, RANDOM())) as AGE,
        CASE WHEN UNIFORM(0, 1, RANDOM()) < 0.6 THEN 'F' ELSE 'M' END as GENDER,
        FLOOR(UNIFORM(1, 15, RANDOM())) as NUM_CONDITIONS,
        FLOOR(UNIFORM(0, 25, RANDOM())) as NUM_MEDICATIONS,
        FLOOR(UNIFORM(1, 200, RANDOM())) as NUM_CLAIMS
    FROM TABLE(GENERATOR(ROWCOUNT => 50000))  -- Generate 50K sample patients
)
SELECT 
    PATIENT_ID,
    AGE,
    GENDER,
    NUM_CONDITIONS,
    NUM_MEDICATIONS, 
    NUM_CLAIMS,
    -- Add some derived healthcare metrics
    CASE 
        WHEN AGE >= 65 AND NUM_CONDITIONS >= 5 THEN 'HIGH_RISK'
        WHEN AGE >= 50 AND NUM_CONDITIONS >= 3 THEN 'MEDIUM_RISK'  
        ELSE 'LOW_RISK'
    END as RISK_CATEGORY,
    CASE WHEN NUM_MEDICATIONS >= 10 THEN 1 ELSE 0 END as POLYPHARMACY_FLAG,
    CURRENT_TIMESTAMP() as CREATED_DATE
FROM patient_base
""").collect()

# Verify the data was created
verification = session.sql("SELECT COUNT(*) as row_count FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.PREPARED_HEALTHCARE_ANALYTICS").collect()
row_count = verification[0]['ROW_COUNT']

print(f"SUCCESS: Created PREPARED_HEALTHCARE_ANALYTICS table with {row_count:,} patient records")
print("This table is now ready for feature engineering in notebook 4")

# Show sample of the data
print("\nSample healthcare analytics data:")
sample_data = session.sql("""
    SELECT PATIENT_ID, AGE, GENDER, NUM_CONDITIONS, NUM_MEDICATIONS, NUM_CLAIMS, RISK_CATEGORY
    FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.PREPARED_HEALTHCARE_ANALYTICS
    LIMIT 5
""").to_pandas()
print(sample_data.to_string(index=False))


## Analytics Infrastructure Complete!

Your ML analytics infrastructure is now ready:

### Analytics Tables (DEMO_ANALYTICS)
- **PATIENT_RISK_SCORES**: Ready for risk assessments
- **DRUG_INTERACTION_ALERTS**: Drug interaction monitoring
- **AE_PREDICTIONS**: Model prediction storage
- **POPULATION_VS_FAERS_COMPARISON**: Population analysis
- **PREPARED_HEALTHCARE_ANALYTICS**: 50K patient records for ML training

### ML Model Tables (ML_MODELS)
- **MODEL_REGISTRY**: Model version tracking
- **FEATURE_IMPORTANCE**: Feature analysis storage
- **MODEL_PREDICTIONS_LOG**: Detailed prediction logging

## Ready for Machine Learning
The foundation is now complete! These tables will support:
- Model training and evaluation with real healthcare data
- Real-time predictions and logging
- Performance monitoring and drift detection
- Business analytics and reporting

## Next Steps
**Run Notebook 3b**: Integrate the FAERS data with synthetic patient data.

---
*Analytics infrastructure + healthcare data provides the complete foundation for enterprise ML operations.*
